# Pydantic

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HF_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from langchain.chat_models import init_chat_model

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq"
)


c:\Users\nithi\Desktop\GitHub\Gen-AI-Course\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pydantic import BaseModel , Field
 
class movie(BaseModel):
    title:str= Field(description="The title of the movie")
    year:int = Field(description="This year the movie was released")
    director:str= Field(description="The director of the movie")
    rating:float= Field(description="The movie rating out of 10")

response  =model.with_structured_output(movie)
print(response)


first=RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001EC22EF1400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EC22EF2120>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movie rating out of 10', 'type': 'number'}}, 'required': ['title'

In [10]:
print(response.invoke("provie the details of the movie  Inception"))

title='Inception' year=2010 director='Christopher Nolan' rating=8.5


# Message Output alongside parsed structure

In [11]:
from pydantic import BaseModel , Field
 
class movie(BaseModel):
    title:str= Field(...,description="The title of the movie")
    year:int = Field(...,description="This year the movie was released")
    director:str= Field(...,description="The director of the movie")
    rating:float= Field(...,description="The movie rating out of 10")

response  =model.with_structured_output(movie, include_raw=True)
# print(response)

print(response.invoke("provie the details of the movie  Inception"))


{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'd9rw2sg9y', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.8,"title":"Inception","year":2010}', 'name': 'movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 282, 'total_tokens': 315, 'completion_time': 0.040955662, 'completion_tokens_details': None, 'prompt_time': 0.016868758, 'prompt_tokens_details': None, 'queue_time': 0.049005871, 'total_time': 0.05782442}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c3181-2452-7a51-9721-20ba27bc7b01-0', tool_calls=[{'name': 'movie', 'args': {'director': 'Christopher Nolan', 'rating': 8.8, 'title': 'Inception', 'year': 2010}, 'id': 'd9rw2sg9y', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 282, 'output_tokens': 33, 'to

# Nested Structure

In [16]:
class actor(BaseModel):
    name: str
    role: str

class movie_details(BaseModel):
    title : str
    year :  int
    cast : list[actor]
    genres  : list[str]
    Budget :  float | None = Field(None , description="badget in million USD")

response  =model.with_structured_output(movie_details)
# print(response)

print(response.invoke("provie the details of the movie  Inception"))

title='Inception' year=2010 cast=[actor(name='Leonardo DiCaprio', role='Cobb'), actor(name='Joseph Gordon-Levitt', role='Arthur'), actor(name='Ellen Page', role='Ariadne'), actor(name='Tom Hardy', role='Eames'), actor(name='Ken Watanabe', role='Saito'), actor(name='Dileep Rao', role='Yusuf'), actor(name='Cillian Murphy', role='Robert Fischer'), actor(name='Tom Berenger', role='Browning'), actor(name='Pete Postlethwaite', role='Maurice Fischer'), actor(name='Marion Cotillard', role="Mal/Cobb's Wife")] genres=['Action', 'Sci-Fi', 'Thriller'] Budget=160.0


# TypesDict

In [29]:
from typing_extensions import TypedDict , Annotated

class MovieDict(TypedDict):
    title:  Annotated[str, ...,"The title of the movie"]
    year : Annotated[int, ...,"This year the movie was released"]
    director: Annotated[str, ...,"The director of the movie"]
    rating : Annotated[float ,...,"The movie rating out of 10"]

In [30]:

response  =model.with_structured_output(MovieDict)
# print(response)

print(response.invoke("provie the details of the movie  Avengers Endgame"))

{'director': 'Anthony Russo', 'rating': 8.7, 'title': 'Avengers: Endgame', 'year': 2019}


# Data Classes

In [38]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_groq import ChatGroq

class Movie(BaseModel):
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie rating out of 10")

model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.environ["GROQ_API_KEY"]
)

agent = create_agent(
    model=model,
    response_format=Movie
)


result  = agent.invoke({
    "messages":[{"role":"user","content" :'Sarathi movie released in 2013 and director is Abi and got the rating of 9 of 10'}
                ]
})

print(result['structured_response'])

title='Sarathi' year=2013 director='Abi' rating=9.0


In [39]:
from dataclasses import dataclass

@dataclass
class Movie(BaseModel):
    title: str
    year: int 
    director:str
    rating: float


model = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.environ["GROQ_API_KEY"]
)

agent = create_agent(
    model=model,
    response_format=Movie
)


result  = agent.invoke({
    "messages":[{"role":"user","content" :'Sarathi movie released in 2013 and director is Abi and got the rating of 9 of 10'}
                ]
})

print(result['structured_response'])

title='Sarathi' year=2013 director='Abi' rating=9.0
